### Merging Customer and Final invoice

* Imports

In [1]:
import numpy as np
import pandas as pd
import re

* Functions

In [2]:
def get_master_dict(df):
    cols = df.columns
    if len(cols) != 2:
        return r'Only 2 columns allowed (key, description)'
    return df.dropna().set_index(cols[0]).to_dict()[cols[1]]

* Code

In [3]:
xls = pd.ExcelFile('data/Customer_Data.xlsx')
customer = pd.read_excel(xls, 'Sheet1')
customer_master = pd.read_excel(xls, 'Sheet2', skiprows=1)

In [4]:
invoice = pd.read_csv('data/invoice_refined.csv', low_memory=False)

In [5]:
invoice.dtypes

Unnamed: 0               int64
Area / Locality         object
CITY                    object
Cash /Cashless Type     object
Claim No.               object
Cust Type               object
Customer No.            object
District                object
Expiry Date             object
Gate Pass Date          object
Gate Pass Time          object
Invoice Date            object
Invoice No               int64
Invoice Time            object
Job Card No              int64
JobCard Date            object
JobCard Time            object
KMs Reading              int64
Labour Total           float64
Make                    object
Misc Total             float64
Model                   object
ODN No.                float64
OSL Total              float64
Order Type              object
Outstanding Amt        float64
Parts Total            float64
Plant                   object
Plant Name1             object
Policy no.              object
Print Status            object
Recovrbl Exp           float64
Regn No 

---

#### Customer data merge

In [6]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555338 entries, 0 to 555337
Data columns (total 9 columns):
Business Partner    555338 non-null object
Customer No.        555338 non-null object
Partner Type        555260 non-null float64
Data Origin         555202 non-null object
Title               509031 non-null float64
Marital Status      16037 non-null float64
Occupation          14558 non-null float64
Date of Birth       13444 non-null object
Death date          663 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 38.1+ MB


In [7]:
customer.shape

(555338, 9)

In [8]:
customer['Customer No.'].nunique()

555338

In [9]:
customer_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 15 columns):
Type              7 non-null float64
Description       7 non-null object
Unnamed: 2        0 non-null float64
Origin            22 non-null object
Description.1     22 non-null object
Short Name        22 non-null object
Unnamed: 6        0 non-null float64
Key               4 non-null float64
Title             4 non-null object
Unnamed: 9        0 non-null float64
Marital Status    5 non-null float64
Short Name.1      5 non-null object
Unnamed: 12       0 non-null float64
Occupation        31 non-null int64
Description.2     31 non-null object
dtypes: float64(7), int64(1), object(7)
memory usage: 3.7+ KB


In [10]:
customer_master = customer_master[['Type', 'Description', 'Origin', 'Description.1',
       'Short Name', 'Key', 'Title', 
       'Marital Status', 'Short Name.1', 'Occupation',
       'Description.2']]

In [11]:
customer_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 11 columns):
Type              7 non-null float64
Description       7 non-null object
Origin            22 non-null object
Description.1     22 non-null object
Short Name        22 non-null object
Key               4 non-null float64
Title             4 non-null object
Marital Status    5 non-null float64
Short Name.1      5 non-null object
Occupation        31 non-null int64
Description.2     31 non-null object
dtypes: float64(3), int64(1), object(7)
memory usage: 2.7+ KB


In [12]:
customer['Partner Type Desc'] = customer['Partner Type'].map(get_master_dict(customer_master.iloc[:,0:2]))

In [13]:
customer['Data Origin Desc'] = customer['Data Origin'].map(get_master_dict(customer_master.iloc[:,2:4]))

In [14]:
customer['Title Desc'] = customer['Title'].map(get_master_dict(customer_master.loc[:,['Key','Title']]))

In [15]:
customer['Marital Status Desc'] = customer['Marital Status'].map(get_master_dict(
    customer_master.loc[:,['Marital Status','Short Name.1']]))

In [16]:
customer['Occupation Desc'] = customer['Occupation'].map(get_master_dict(
    customer_master.loc[:,['Occupation','Description.2']]))

In [17]:
desc_list = ['Partner Type Desc','Data Origin Desc','Title Desc','Marital Status Desc','Occupation Desc']
for col in desc_list:
    print(col)
    print(customer[col].value_counts(dropna=False))
    print('---\n')

Partner Type Desc
Retail               554967
NaN                      78
Corporate                68
Fleet                    62
Insurance Company        57
Contact Person           56
Employee                 48
Surveyor                  2
Name: Partner Type Desc, dtype: int64
---

Data Origin Desc
Reference - Customer                149861
Reference - Employee                121660
Camp - Outdoor                       63237
Camp - Workshop                      38326
Fleet                                36801
Spotted the outlet                   29780
Other outdoor activity               28828
Outdoor Sales Activity               15382
Hoardings/Outdoor Advertisements     13967
Just Dial/Other helpline             12122
Newspaper AD                          8118
Float activity                        8023
Company website                       6260
Newspaper leaflet                     5685
Emailers                              5082
Insurance Company                     4014
Petrol pum

In [18]:
# customer.to_csv('data/customer_desc.csv')

In [21]:
customer['Customer No.'] = customer['Customer No.'].astype(str)
cust_custno_list = customer['Customer No.'].unique().tolist()

In [22]:
invoice['Customer No.'] = invoice['Customer No.'].astype(str)
inv_custno_list = invoice['Customer No.'].unique().tolist()

In [32]:
#check if all customers present in customer data
custno_not_in_both = [cno for cno in inv_custno_list if cno not in cust_custno_list]
custno_not_in_both

['188602', '188547']

In [26]:
len(custno_not_in_both)

301856

In [34]:
invoice[invoice['Customer No.'].str.contains("188602")]


,Unnamed: 0,Area / Locality,CITY,Cash /Cashless Type,Claim No.,Cust Type,Customer No.,District,Expiry Date,Gate Pass Date,...,Recovrbl Exp,Regn No,Technician Name,Total Amt Wtd Tax.,Total Value,User ID,plant_city,plant_state,invoice_month,invoice_year
369708,369708,G.T.ROAD,ALIGARH,NaN,NaN,Retail,188602,Uttar Pradesh,2016-06-12 00:00:00,NaN,...,0.0,UP81AL5265,NaN,1175.53,0.0,X224SA1,ALIGARH,Uttar Pradesh,5,2016


In [35]:

invoice[invoice['Customer No.'].str.contains("188547")]

,Unnamed: 0,Area / Locality,CITY,Cash /Cashless Type,Claim No.,Cust Type,Customer No.,District,Expiry Date,Gate Pass Date,...,Recovrbl Exp,Regn No,Technician Name,Total Amt Wtd Tax.,Total Value,User ID,plant_city,plant_state,invoice_month,invoice_year
369754,369754,AWAS VIKAS,RUDRAPUR,NaN,NaN,Retail,188547,Uttarakhand,2016-09-20 00:00:00,NaN,...,0.0,UK06Y2750,NaN,1135.49,0.0,X233SA1,RUDRAPUR,Uttarakhand,5,2016


In [36]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555338 entries, 0 to 555337
Data columns (total 14 columns):
Business Partner       555338 non-null object
Customer No.           555338 non-null object
Partner Type           555260 non-null float64
Data Origin            555202 non-null object
Title                  509031 non-null float64
Marital Status         16037 non-null float64
Occupation             14558 non-null float64
Date of Birth          13444 non-null object
Death date             663 non-null datetime64[ns]
Partner Type Desc      555260 non-null object
Data Origin Desc       555202 non-null object
Title Desc             509031 non-null object
Marital Status Desc    16037 non-null object
Occupation Desc        14558 non-null object
dtypes: datetime64[ns](1), float64(4), object(9)
memory usage: 59.3+ MB


In [41]:
# null values percentage -dob
customer['Date of Birth'].isnull().sum()/customer.shape[0]*100

97.57913198808653

In [42]:
# null values percentage -death date
customer['Death date'].isnull().sum()/customer.shape[0]*100

99.88061324814798

In [44]:
# null values percentage -Occupation
customer['Occupation'].isnull().sum()/customer.shape[0]*100

97.37853343369264

In [51]:
inv_cust_merged = pd.merge(invoice,customer.drop(['Date of Birth','Death date',
                                                  'Occupation'],axis=1),on='Customer No.',how='left')

In [53]:
inv_cust_merged.shape

(492314, 51)

In [56]:
inv_cust_merged.to_csv('data/invoice_customer_merged.csv')